In [7]:
from math import cos, exp, pi, tanh, sqrt
from scipy.integrate import quad

In [8]:
def phi(x):
    return tanh(x)

def normDist(z):
    return (1/sqrt(2*pi))*exp(-(z**2)/2)


In [20]:
def qSSnext(sigmaW, sigmaB, qAAprev):

    def f(z):
        return (phi(sqrt(qAAprev)*z)**2) * normDist(z)

    intergral = quad(f, -15, 15)
    return (sigmaW**2) * intergral[0] + (sigmaB**2)


def qSSstar(sigmaW, sigmaB, qAAfrst):
    qAAtemp = qAAfrst
    for i in range(30):
        qAAtemp = qAAnext(sigmaW, sigmaB, qAAtemp)

    return qAAtemp

In [19]:
def qAB_next(sigmaW, sigmaB, qAA_prev, qBB_prev, c_prev):
    

12.604149497844299